# Load Data

In [ ]:
import pandas as pd
from datasets import load_dataset

ds = load_dataset("Hello-SimpleAI/HC3", "all")

# Extract and label data for the Hugging Face dataset
rows = []
for entry in ds["train"]:
    for a in entry["chatgpt_answers"]:
        rows.append({"text": a.strip(), "label": 0})  # AI
    for a in entry["human_answers"]:
        rows.append({"text": a.strip(), "label": 1})  # Human

df = pd.DataFrame(rows).dropna()
print(f"Samples: {len(df)}")
df.head()

In [ ]:
# Balance classes
min_class = df['label'].value_counts().min()
data_balanced = pd.concat([
    df[df['label'] == 0].sample(min_class),
    df[df['label'] == 1].sample(min_class)
]).sample(frac=1).reset_index(drop=True)

# Model Stuff

In [ ]:
# Tokenization + Dataset Prep
from transformers import T5Tokenizer, T5ForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch

tokenizer = T5Tokenizer.from_pretrained("t5-small")

class CombinedDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_len)
        self.labels = labels

    def __len__(self): return len(self.labels)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = CombinedDataset(data_balanced['text'].tolist(), data_balanced['label'].tolist(), tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [ ]:
model = T5ForSequenceClassification.from_pretrained("t5-small", num_labels=2)

In [ ]:
# Train

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

from transformers import AdamW
from tqdm import tqdm

optimizer = AdamW(model.parameters(), lr=2e-5)

model.train()
for epoch in range(1):
    total_loss = 0
    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch + 1} Loss: {total_loss:.4f}")

In [ ]:
model.save_pretrained("initial-t5-ai-detector")
tokenizer.save_pretrained("initial-t5-ai-detector")

# Eval

In [ ]:
from transformers import T5ForSequenceClassification, T5Tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and tokenizer from the saved directory
model = T5ForSequenceClassification.from_pretrained("initial-t5-ai-detector").to(device)
tokenizer = T5Tokenizer.from_pretrained("initial-t5-ai-detector")

# Predict
model.eval()
preds = []

In [ ]:
import os
import jsonlines
import torch
from sklearn.metrics import classification_report
from tqdm import tqdm

dev_dir = "cs162-final-dev-main"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
human_texts, machine_texts = [], []

for filename in os.listdir(dev_dir):
    if filename.endswith(".jsonl"):
        file_path = os.path.join(dev_dir, filename)
        with jsonlines.open(file_path) as reader:
            for row in reader:
                human_texts.append(row["human_text"])
                machine_texts.append(row["machine_text"])

test_texts = human_texts + machine_texts
true_labels = [1] * len(human_texts) + [0] * len(machine_texts)

# Predict
model.eval()
preds = []

with torch.no_grad():
    for i in tqdm(range(0, len(test_texts), 8)):
        batch = test_texts[i:i+8]
        encodings = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=256)
        input_ids = encodings["input_ids"].to(device)
        attention_mask = encodings["attention_mask"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        batch_preds = torch.argmax(outputs.logits, dim=1).tolist()
        preds.extend(batch_preds)

# Evaluate
from sklearn.metrics import classification_report

print(classification_report(true_labels, preds, target_names=["AI-generated", "Human-written"]))